# OpenAI Agent SDK

## Agents
Agents are the core building block in your apps. An agent is a large language model (LLM), configured with instructions and tools.

### Basic configuration
The most common properties of an agent you'll configure are:

- **instructions**: also known as a developer message or system prompt.
- **model**: which LLM to use, and optional model_settings to configure model tuning parameters like temperature, top_p, etc.
- **tools**: Tools that the agent can use to achieve its tasks.


```python
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="o3-mini",
    tools=[get_weather],
)
````

## Setup

In [1]:
!pip install openai-agents -q

In [2]:
import os
import getpass
from openai import OpenAI

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [4]:
_set_env("OPENAI_API_KEY")

In [5]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

client

### Example: Basic Agent

In [6]:
from agents import Agent, Runner

agent = Agent(
    model="gpt-4o",
    name="basic_agent",
    instructions="You are a helpful agent."
)

result = await Runner.run(agent, "What's the capital of the USA?")
print(result.final_output)
# The capital of the United States is Washington, D.C.


The capital of the United States is Washington, D.C.


In [7]:
from agents import Agent, Runner

# Including this only if in jupyter notebook environment
import nest_asyncio
nest_asyncio.apply()

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

# Code within the code,
# Functions calling themselves,
# Infinite loop's dance.

Code within the code,  
Endless loops of logic spin—  
Echoes in the shell.


### More Agent Examples

Additional agents can be defined in the same way. `handoff_descriptions` provide additional context for determining handoff routing

In [8]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

### Define your handoffs
On each agent, you can define an inventory of outgoing handoff options that the agent can choose from to decide how to make progress on their task.

In [9]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

### Run the agent orchestration
Let's check that the workflow runs and the triage agent correctly routes between the two specialist agents.

In [10]:
async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")
    print(result.final_output)

In [13]:
if __name__ == "__main__":
    await main()

The capital of France is Paris. This city has been a crucial center of culture, politics, and history for centuries. Paris has played an important role in many historical events, from the French Revolution to being a significant city in both World Wars. It's known for iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.


### Add a guardrail
You can define custom guardrails to run on the input or output.

In [14]:
from agents import GuardrailFunctionOutput, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

### Put it all together
Let's put it all together and run the entire workflow, using handoffs and the input guardrail.



In [16]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio
from IPython.display import Markdown, display

# Including this only if in jupyter notebook environment
import nest_asyncio
nest_asyncio.apply()

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        # print(result.final_output)
        display(Markdown(result.final_output))

        result = await Runner.run(triage_agent, "How can I solve 8x + 7 = -23?")
        # print(result.final_output)
        display(Markdown(result.final_output))

        result = await Runner.run(triage_agent, "what is life")
        # print(result.final_output)
        display(Markdown(result.final_output))
    except Exception as e:
        display(Markdown(str(e)))

if __name__ == "__main__":
    asyncio.run(main())

The first president of the United States was George Washington. He served from 1789 to 1797. Washington was unanimously elected by the Electoral College and is often called the "Father of His Country" for his leadership in the founding of the nation. He played a crucial role as the commander-in-chief of the Continental Army during the American Revolutionary War, and later presided over the Constitutional Convention of 1787, which led to the drafting of the U.S. Constitution. Washington set many precedents for the new nation, including the tradition of a two-term presidency.

Sure! To solve the equation \(8x + 7 = -23\), you'll want to isolate \(x\) on one side of the equation. Let's go through the steps:

### Step 1: Eliminate the constant term from the left side

The equation starts as:

\[ 8x + 7 = -23 \]

**Subtract 7 from both sides** to eliminate the constant term on the left:

\[ 8x + 7 - 7 = -23 - 7 \]

This simplifies to:

\[ 8x = -30 \]

### Step 2: Solve for \(x\)

Now, divide both sides by 8 to solve for \(x\):

\[ \frac{8x}{8} = \frac{-30}{8} \]

This simplifies to:

\[ x = -\frac{30}{8} \]

### Step 3: Simplify the fraction

Simplify \(-\frac{30}{8}\) by dividing the numerator and the denominator by their greatest common divisor, which is 2:

\[ x = -\frac{30 \div 2}{8 \div 2} = -\frac{15}{4} \]

Thus, the solution to the equation \(8x + 7 = -23\) is:

\[ x = -\frac{15}{4} \]

### Example

If you need to verify the solution, substitute \(-\frac{15}{4}\) back into the original equation:

\[ 8\left(-\frac{15}{4}\right) + 7 \]

\[ = -30 + 7 \]

\[ = -23 \]

This checks out, confirming that \(x = -\frac{15}{4}\) is correct.

Guardrail InputGuardrail triggered tripwire

### Functions example

In [8]:
import asyncio

from agents import Agent, Runner, function_tool


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather agent.",
    tools=[get_weather],
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

The weather in Tokyo is sunny.


#### Workshop Project

https://www.youtube.com/watch?v=yCPSj6lfx-0

https://github.com/trancethehuman/ai-workshop-code/tree/main/projects/openai-agents-sdk